In [28]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sklearn as skl
import tensorflow as tf
from pathlib import Path
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [29]:
#import data

# Create Engine
engine = create_engine("sqlite:///../lifestyle_sleep_data.sqlite")
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

# Save reference to the table
Sleep = Base.classes.lifestyle_sleep_data


In [30]:
# Create our session (link) from Python to the DB
session = Session(engine)
conn = engine.connect()

In [31]:
# Query All Records in the the Database
sleep_df = pd.read_sql("SELECT * FROM lifestyle_sleep_data", conn)
sleep_df=sleep_df.drop(columns = "person_id")
sleep_df.head()

,gender,age,occupation,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,BMI_category,heart_rate,daily_steps,sleep_disorder,systolic_blood_pressure,diastolic_blood_pressure
0,Male,27,Software Engineer,6.1,6,42,6,Overweight,77,4200,None,126,83
1,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,None,125,80
2,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,None,125,80
3,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90
4,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90


In [32]:
#temporary direct import of data while waiting for sql database update
#sleep_df=pd.read_csv("../Data/Sleep_health_mk1.csv")
#handle lower case/upper case descrepancy
#sleep_df.columns=sleep_df.columns.str.strip().str.lower()
#convert spaces to underscore
#sleep_df.columns = sleep_df.columns.str.replace(' ', '_')
#remove unnamed column:
#sleep_df.drop(columns="unnamed:_0",inplace=True)
#restore caps to BMI
#sleep_df=sleep_df.rename(columns={"bmi_category":"BMI_category"})
#sleep_df.head()

Data Preprocessing:

In [33]:
# Verify the categories of the "occupation" column
occupations=sleep_df['occupation'].value_counts()
occupations

Nurse                   73
Doctor                  71
Engineer                63
Lawyer                  47
Teacher                 40
Accountant              37
Salesperson             32
Software Engineer        4
Scientist                4
Sales Representative     2
Manager                  1
Name: occupation, dtype: int64

In [34]:
#bin occupations with fewer than 30 subjects
#determine which occupations qualify and add to a list
occupations_replace=[]
for occ in occupations.items():
    if occ[1]<30:
        occupations_replace.append(occ[0])
#replace said occupations with "Other"
for occ in occupations_replace:
    sleep_df['occupation']=sleep_df['occupation'].replace(occ,"Other")
#verify successful binning
sleep_df['occupation'].value_counts()

Nurse          73
Doctor         71
Engineer       63
Lawyer         47
Teacher        40
Accountant     37
Salesperson    32
Other          11
Name: occupation, dtype: int64

In [35]:
len(sleep_df['occupation'].value_counts())

8

In [9]:
# Transform the occupation column using get_dummies
occupation_dummies = pd.get_dummies(sleep_df['occupation'])

# Concatenate the sleep_df and the occupation_dummies DataFrames
sleep_df = pd.concat([sleep_df, occupation_dummies], axis=1)

# Drop the original occupation column
sleep_df =sleep_df.drop(columns=["occupation"])

# Display the DataFrame
sleep_df.head()

,gender,age,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,BMI_category,heart_rate,daily_steps,sleep_disorder,systolic_blood_pressure,diastolic_blood_pressure,Accountant,Doctor,Engineer,Lawyer,Nurse,Other,Salesperson,Teacher
0,Male,27,6.1,6,42,6,Overweight,77,4200,None,126,83,0,0,0,0,0,1,0,0
1,Male,28,6.2,6,60,8,Normal,75,10000,None,125,80,0,1,0,0,0,0,0,0
2,Male,28,6.2,6,60,8,Normal,75,10000,None,125,80,0,1,0,0,0,0,0,0
3,Male,28,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90,0,0,0,0,0,1,0,0
4,Male,28,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90,0,0,0,0,0,1,0,0


In [10]:
# Clean-up the "BMI-category"
sleep_df["BMI_category"] = sleep_df["BMI_category"].replace({'Normal': 'Normal Weight', 'Obese': 'Overweight'})
sleep_df['BMI_category'].value_counts()

Normal Weight    216
Overweight       158
Name: BMI_category, dtype: int64

In [11]:
# Transform the "BMI_category" column using get_dummies
bmi_dummies = pd.get_dummies(sleep_df['BMI_category'])

# Concatenate the sleep_df and the bmi_dummies DataFrames
sleep_df = pd.concat([sleep_df, bmi_dummies], axis=1)

# Drop the original "BMI_category"  column
sleep_df =sleep_df.drop(columns=["BMI_category"])

# Display the DataFrame
sleep_df.head()

,gender,age,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,heart_rate,daily_steps,sleep_disorder,systolic_blood_pressure,...,Accountant,Doctor,Engineer,Lawyer,Nurse,Other,Salesperson,Teacher,Normal Weight,Overweight
0,Male,27,6.1,6,42,6,77,4200,None,126,...,0,0,0,0,0,1,0,0,0,1
1,Male,28,6.2,6,60,8,75,10000,None,125,...,0,1,0,0,0,0,0,0,1,0
2,Male,28,6.2,6,60,8,75,10000,None,125,...,0,1,0,0,0,0,0,0,1,0
3,Male,28,5.9,4,30,8,85,3000,Sleep Apnea,140,...,0,0,0,0,0,1,0,0,0,1
4,Male,28,5.9,4,30,8,85,3000,Sleep Apnea,140,...,0,0,0,0,0,1,0,0,0,1


In [12]:
# Verify the values of the "gender" column
sleep_df['gender'].value_counts()

Male      189
Female    185
Name: gender, dtype: int64

In [13]:
# Transform the "gender" column using get_dummies
gender_dummies = pd.get_dummies(sleep_df['gender'])

# Concatenate the sleep_df and the gender_dummies DataFrames
sleep_df = pd.concat([sleep_df, gender_dummies], axis=1)

# Drop the original "gender" column
sleep_df =sleep_df.drop(columns=["gender"])

# Display the DataFrame
sleep_df.head()

,age,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,heart_rate,daily_steps,sleep_disorder,systolic_blood_pressure,diastolic_blood_pressure,...,Engineer,Lawyer,Nurse,Other,Salesperson,Teacher,Normal Weight,Overweight,Female,Male
0,27,6.1,6,42,6,77,4200,None,126,83,...,0,0,0,1,0,0,0,1,0,1
1,28,6.2,6,60,8,75,10000,None,125,80,...,0,0,0,0,0,0,1,0,0,1
2,28,6.2,6,60,8,75,10000,None,125,80,...,0,0,0,0,0,0,1,0,0,1
3,28,5.9,4,30,8,85,3000,Sleep Apnea,140,90,...,0,0,0,1,0,0,0,1,0,1
4,28,5.9,4,30,8,85,3000,Sleep Apnea,140,90,...,0,0,0,1,0,0,0,1,0,1


In [14]:
# Verify the values of the "sleep_disorder" column
sleep_df['sleep_disorder'].value_counts()

None           219
Sleep Apnea     78
Insomnia        77
Name: sleep_disorder, dtype: int64

In [15]:
#diverge sheets with one preserving Sleep Apnea and Insomnia, vs combining into a single bin
sleep_df2=sleep_df.copy()

In [16]:
# Clean-up the "sleep_disorder" column
sleep_df["sleep_disorder"] = sleep_df["sleep_disorder"].replace({'Sleep Apnea': 'Sleep Disorder', 'Insomnia': 'Sleep Disorder'})
# Verify the values of the "sleep_disorder" column
sleep_df['sleep_disorder'].value_counts()

None              219
Sleep Disorder    155
Name: sleep_disorder, dtype: int64

In [17]:
# Encoding the sleep_disorder column using a custom function
def encode_disorder(disorder):
    """
    This function encodes sleep disorder status by setting sleep apnea as 1 and no as 0.
    """
    if disorder == "Sleep Disorder":
        return 1
    else:
        return 0

# Call the encode_marriage function on the marriage column
sleep_df["sleep_disorder"] = sleep_df["sleep_disorder"].apply(encode_disorder)

# Review the DataFrame 
sleep_df.head()

,age,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,heart_rate,daily_steps,sleep_disorder,systolic_blood_pressure,diastolic_blood_pressure,...,Engineer,Lawyer,Nurse,Other,Salesperson,Teacher,Normal Weight,Overweight,Female,Male
0,27,6.1,6,42,6,77,4200,0,126,83,...,0,0,0,1,0,0,0,1,0,1
1,28,6.2,6,60,8,75,10000,0,125,80,...,0,0,0,0,0,0,1,0,0,1
2,28,6.2,6,60,8,75,10000,0,125,80,...,0,0,0,0,0,0,1,0,0,1
3,28,5.9,4,30,8,85,3000,1,140,90,...,0,0,0,1,0,0,0,1,0,1
4,28,5.9,4,30,8,85,3000,1,140,90,...,0,0,0,1,0,0,0,1,0,1


In [18]:
# Transform the "sleep_disorder" column using get_dummies
disorder_dummies = pd.get_dummies(sleep_df2['sleep_disorder'])

# Identify target columns for later
target_col=list(disorder_dummies.columns)

# Concatenate the sleep_df and the disorder_dummies DataFrames
sleep_df2 = pd.concat([sleep_df2, disorder_dummies], axis=1)

# Drop the original "sleep_disorder" column
sleep_df2 = sleep_df2.drop(columns=["sleep_disorder"])

# Display the DataFrame
sleep_df2.head()

,age,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,heart_rate,daily_steps,systolic_blood_pressure,diastolic_blood_pressure,Accountant,...,Other,Salesperson,Teacher,Normal Weight,Overweight,Female,Male,Insomnia,None,Sleep Apnea
0,27,6.1,6,42,6,77,4200,126,83,0,...,1,0,0,0,1,0,1,0,1,0
1,28,6.2,6,60,8,75,10000,125,80,0,...,0,0,0,1,0,0,1,0,1,0
2,28,6.2,6,60,8,75,10000,125,80,0,...,0,0,0,1,0,0,1,0,1,0
3,28,5.9,4,30,8,85,3000,140,90,0,...,1,0,0,0,1,0,1,0,0,1
4,28,5.9,4,30,8,85,3000,140,90,0,...,1,0,0,0,1,0,1,0,0,1


In [19]:
target_col

['Insomnia', 'None', 'Sleep Apnea']

Neural Network Time

In [20]:
# Seperate the features, X,  from the target variable, y
y = sleep_df['sleep_disorder']
X = sleep_df.drop(columns='sleep_disorder')

In [22]:

print(X.columns)
keys=list(X.columns)

Index(['age', 'sleep_duration', 'quality_of_sleep', 'physical_activity_level',
       'stress_level', 'heart_rate', 'daily_steps', 'systolic_blood_pressure',
       'diastolic_blood_pressure', 'Accountant', 'Doctor', 'Engineer',
       'Lawyer', 'Nurse', 'Other', 'Salesperson', 'Teacher', 'Normal Weight',
       'Overweight', 'Female', 'Male'],
      dtype='object')


In [39]:
X.iloc[0]

age                           27.0
sleep_duration                 6.1
quality_of_sleep               6.0
physical_activity_level       42.0
stress_level                   6.0
heart_rate                    77.0
daily_steps                 4200.0
systolic_blood_pressure      126.0
diastolic_blood_pressure      83.0
Accountant                     0.0
Doctor                         0.0
Engineer                       0.0
Lawyer                         0.0
Nurse                          0.0
Other                          1.0
Salesperson                    0.0
Teacher                        0.0
Normal Weight                  0.0
Overweight                     1.0
Female                         0.0
Male                           1.0
Name: 0, dtype: float64

In [40]:
X.iloc[0].to_numpy()

array([2.70e+01, 6.10e+00, 6.00e+00, 4.20e+01, 6.00e+00, 7.70e+01,
       4.20e+03, 1.26e+02, 8.30e+01, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       1.00e+00, 0.00e+00, 1.00e+00])

In [26]:
for key in keys:
    max_=X[key].max()
    min_=X[key].min()
    print(f"{key}: max:{max_}, min:{min_}, range:{(max_-min_)}")

age: max:59, min:27, range:32
sleep_duration: max:8.5, min:5.8, range:2.7
quality_of_sleep: max:9, min:4, range:5
physical_activity_level: max:90, min:30, range:60
stress_level: max:8, min:3, range:5
heart_rate: max:86, min:65, range:21
daily_steps: max:10000, min:3000, range:7000
systolic_blood_pressure: max:142, min:115, range:27
diastolic_blood_pressure: max:95, min:75, range:20
Accountant: max:1, min:0, range:1
Doctor: max:1, min:0, range:1
Engineer: max:1, min:0, range:1
Lawyer: max:1, min:0, range:1
Nurse: max:1, min:0, range:1
Other: max:1, min:0, range:1
Salesperson: max:1, min:0, range:1
Teacher: max:1, min:0, range:1
Normal Weight: max:1, min:0, range:1
Overweight: max:1, min:0, range:1
Female: max:1, min:0, range:1
Male: max:1, min:0, range:1


In [22]:
#split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [23]:
X_train.head()

,age,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,heart_rate,daily_steps,systolic_blood_pressure,diastolic_blood_pressure,Accountant,...,Engineer,Lawyer,Nurse,Other,Salesperson,Teacher,Normal Weight,Overweight,Female,Male
59,32,7.7,7,75,6,70,8000,120,80,0,...,0,0,0,0,0,0,1,0,0,1
12,29,6.1,6,30,8,70,8000,120,80,0,...,0,0,0,0,0,0,1,0,0,1
289,50,6.1,6,90,8,75,10000,140,95,0,...,0,0,1,0,0,0,0,1,1,0
201,43,7.8,8,90,5,70,8000,130,85,0,...,1,0,0,0,0,0,1,0,0,1
91,35,7.3,8,60,4,65,5000,125,80,0,...,1,0,0,0,0,0,1,0,0,1


In [24]:
#Scale X
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 176       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 227 (908.00 Byte)
Trainable params: 227 (908.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100


9/9 [==============================] - 1s 2ms/step - loss: 0.6294 - accuracy: 0.5679
Epoch 2/100
9/9 [==============================] - 0s 2ms/step - loss: 0.5919 - accuracy: 0.6964
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 0.5606 - accuracy: 0.7750
Epoch 4/100
9/9 [==============================] - 0s 2ms/step - loss: 0.5334 - accuracy: 0.7964
Epoch 5/100
9/9 [==============================] - 0s 2ms/step - loss: 0.5117 - accuracy: 0.7964
Epoch 6/100
9/9 [==============================] - 0s 2ms/step - loss: 0.4907 - accuracy: 0.8071
Epoch 7/100
9/9 [==============================] - 0s 2ms/step - loss: 0.4713 - accuracy: 0.8893
Epoch 8/100
9/9 [==============================] - 0s 2ms/step - loss: 0.4522 - accuracy: 0.9143
Epoch 9/100
9/9 [==============================] - 0s 2ms/step - loss: 0.4355 - accuracy: 0.9214
Epoch 10/100
9/9 [==============================] - 0s 2ms/step - loss: 0.4195 - accuracy: 0.9286
Epoch 11/100
9/9 [=======================

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3/3 - 0s - loss: 0.3379 - accuracy: 0.9149 - 202ms/epoch - 67ms/step
Loss: 0.33785295486450195, Accuracy: 0.914893627166748


In [29]:
# Export our model to HDF5 file
nn.save_weights('sleep_model.hdf5')

Try again without the sleep disorders binned

In [30]:
# Seperate the features, X,  from the target variable, y
y2 = sleep_df2[target_col]
X2 = sleep_df2.drop(columns=target_col)

In [31]:
#split dataset
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=1)

In [32]:
#Scale X
scaler2 = StandardScaler()

# Fit the StandardScaler
X_scaler2 = scaler2.fit(X_train2)

# Scale the data
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features2 = len(X_train2.columns)
hidden_nodes2_layer1 =  8
hidden_nodes2_layer2 = 5

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes2_layer1, input_dim=number_input_features2, activation="relu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes2_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=3, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 176       
                                                                 
 dense_4 (Dense)             (None, 5)                 45        
                                                                 
 dense_5 (Dense)             (None, 3)                 18        
                                                                 
Total params: 239 (956.00 Byte)
Trainable params: 239 (956.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [35]:
# Train the model
fit_model2 = nn2.fit(X_train_scaled2,y_train2,epochs=100)

Epoch 1/100


9/9 [==============================] - 1s 2ms/step - loss: 0.5994 - accuracy: 0.5929
Epoch 2/100
9/9 [==============================] - 0s 2ms/step - loss: 0.5697 - accuracy: 0.5964
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 0.5439 - accuracy: 0.5964
Epoch 4/100
9/9 [==============================] - 0s 2ms/step - loss: 0.5198 - accuracy: 0.6036
Epoch 5/100
9/9 [==============================] - 0s 2ms/step - loss: 0.5002 - accuracy: 0.6036
Epoch 6/100
9/9 [==============================] - 0s 2ms/step - loss: 0.4813 - accuracy: 0.6036
Epoch 7/100
9/9 [==============================] - 0s 2ms/step - loss: 0.4636 - accuracy: 0.6214
Epoch 8/100
9/9 [==============================] - 0s 2ms/step - loss: 0.4475 - accuracy: 0.6607
Epoch 9/100
9/9 [==============================] - 0s 2ms/step - loss: 0.4338 - accuracy: 0.6679
Epoch 10/100
9/9 [==============================] - 0s 2ms/step - loss: 0.4206 - accuracy: 0.7250
Epoch 11/100
9/9 [=======================

In [36]:
# Evaluate the model using the test data
model_loss2, model_accuracy2 = nn2.evaluate(X_test_scaled2,y_test2,verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

3/3 - 0s - loss: 0.3034 - accuracy: 0.8936 - 166ms/epoch - 55ms/step
Loss: 0.30335119366645813, Accuracy: 0.8936170339584351


In [37]:
# Export our model to HDF5 file
nn2.save_weights('sleep_model2.hdf5')